In [1]:
%load_ext autoreload
%autoreload 2

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import numpy as np

if torch.cuda.is_available():
    print("using cuda")
    

using cuda


In [2]:
from data.shapenet import ShapeNetVox

# Create a dataset with train split
trainset = ShapeNetVox('all')
# valset = ShapeNetVox('val')


In [3]:
from util.visualization import visualize_occupancy

shape_data = trainset[2]
print(f'Name: {shape_data["name"]}')  # expected output: 04379243/d120d47f8c9bc5028640bc5712201c4a
print(f'Voxel Dimensions: {shape_data["voxel"].shape}')  # expected output: (1, 32, 32, 32)
print(f'Label: {shape_data["label"]} | {ShapeNetVox.classes[shape_data["label"]]}')  # expected output: 10, 04379243

visualize_occupancy(shape_data["voxel"].squeeze(), flip_axes=True)

Name: 02691156/97c12e6155fdf8ca90baeef8ba5b93e5
Voxel Dimensions: (1, 32, 32, 32)
Label: 0 | 02691156


Output()

In [4]:
from model import _G, _D
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 256

model_params = {"cube_len" : 32,
               "z_size": 256}

train_loader = DataLoader(trainset,  batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(valset,  batch_size=batch_size, shuffle=True)

G = _G(model_params)
D = _D(model_params)

G.to(device)
D.to(device)

D_optim = optim.Adam(D.parameters(), lr=1e-3)
G_optim = optim.Adam(G.parameters(), lr=1e-3)

loss_fn = nn.BCELoss()


In [5]:
num_epochs = 50

for epoch in range(num_epochs):
    d_losses = []
    g_losses = []
    d_acc = []
    for i, batch in enumerate(train_loader):
        batch_gpu = batch["voxel"].to(device)

#             if batch.size()[0] != int(args.batch_size):
#                 #print("batch_size != {} drop last incompatible batch".format(int(args.batch_size)))
#                 continue

        Z = torch.Tensor(batch_gpu.shape[0], model_params["z_size"]).normal_(0, 0.33).to(device)
        real_labels = torch.ones((batch_gpu.shape[0], )).to(device)
        fake_labels = torch.zeros((batch_gpu.shape[0], )).to(device)

        # ============= Train the discriminator =============#
        d_real = D(batch_gpu)
#         print(d_real.shape, real_labels.shape)
        d_real = torch.squeeze(d_real)
        d_real_loss = loss_fn(d_real, real_labels)


        fake = G(Z)
        d_fake = D(fake)
        d_fake = torch.squeeze(d_fake)
        d_fake_loss = loss_fn(d_fake, fake_labels)

        d_loss = d_real_loss + d_fake_loss
        d_losses.append(d_loss.item())

        d_real_acu = torch.ge(d_real.squeeze(), 0.5).float()
        d_fake_acu = torch.le(d_fake.squeeze(), 0.5).float()
        d_total_acu = torch.mean(torch.cat((d_real_acu, d_fake_acu),0))
        d_acc.append(d_total_acu.item())

        if d_total_acu <= 0.6:
            D.zero_grad()
            d_loss.backward()
            D_optim.step()

        # =============== Train the generator ===============#

        Z = torch.Tensor(batch_gpu.shape[0], model_params["z_size"]).normal_(0, 0.33).to(device)

        fake = G(Z)
        d_fake = D(fake)
        d_fake = torch.squeeze(d_fake)
        g_loss = loss_fn(d_fake, real_labels)
        g_losses.append(g_loss.item())

        D.zero_grad()
        G.zero_grad()
        g_loss.backward()
        G_optim.step()
        if i%4 == 0:
            print(f"Epoch: {epoch}, iteration: {i}, d_real_acu: {torch.mean(d_real_acu)}, d_fake_acu: {torch.mean(d_fake_acu)}, d_total_acu: {d_total_acu}")
            print(f"d_real_loss: {d_real_loss}, d_fake_loss: {d_fake_loss}, g_loss: {g_loss}")
            visualize_occupancy((fake[0].detach().cpu().numpy().squeeze() > 0.5).astype(int), flip_axes=True)
        
    
    torch.save(G.state_dict(), "output_models/G_{}".format(epoch))
    torch.save(D.state_dict(), "output_models/D_{}".format(epoch))
    epoch_dloss = np.mean(np.array(d_losses))
    epoch_gloss = np.mean(np.array(g_losses))
    epoch_dacc = np.mean(np.array(d_acc))
    print(f"Epoch: {epoch}, d_loss: {epoch_dloss}, g_loss: {epoch_gloss}, dacc: {epoch_dacc}")
    


Epoch: 0, iteration: 0, d_real_acu: 0.15625, d_fake_acu: 0.96484375, d_total_acu: 0.560546875
d_real_loss: 0.7483044862747192, d_fake_loss: 0.5993783473968506, g_loss: 3.775512218475342


Output()

Epoch: 0, iteration: 4, d_real_acu: 0.9765625, d_fake_acu: 1.0, d_total_acu: 0.98828125
d_real_loss: 0.43182891607284546, d_fake_loss: 0.25553983449935913, g_loss: 1.4962002038955688


Output()

Epoch: 0, iteration: 8, d_real_acu: 0.984375, d_fake_acu: 1.0, d_total_acu: 0.9921875
d_real_loss: 0.41463953256607056, d_fake_loss: 0.5934953689575195, g_loss: 0.8042757511138916


...

Epoch: 49, iteration: 16, d_real_acu: 0.57421875, d_fake_acu: 0.23046875, d_total_acu: 0.40234375
d_real_loss: 0.6510553359985352, d_fake_loss: 0.7607672214508057, g_loss: 0.7436473369598389


Output()

Epoch: 49, iteration: 20, d_real_acu: 0.6941176652908325, d_fake_acu: 0.42941176891326904, d_total_acu: 0.5617647171020508
d_real_loss: 0.6353140473365784, d_fake_loss: 0.7317985892295837, g_loss: 0.742130696773529


Output()

Epoch: 49, d_loss: 1.3323855854216076, g_loss: 0.7364028152965364, dacc: 0.562650998433431


In [19]:
!python train.py

using cuda
Epoch: 0
100%|███████████████████████████████████████████| 83/83 [00:28<00:00,  2.89it/s]
Epoch: 1
100%|███████████████████████████████████████████| 83/83 [00:25<00:00,  3.23it/s]
Epoch: 2
100%|███████████████████████████████████████████| 83/83 [00:24<00:00,  3.35it/s]
Epoch: 3
100%|███████████████████████████████████████████| 83/83 [00:25<00:00,  3.19it/s]
Epoch: 4
100%|███████████████████████████████████████████| 83/83 [00:25<00:00,  3.26it/s]
Epoch: 5
100%|███████████████████████████████████████████| 83/83 [00:25<00:00,  3.23it/s]
Epoch: 6
100%|███████████████████████████████████████████| 83/83 [00:25<00:00,  3.27it/s]
Epoch: 7
100%|███████████████████████████████████████████| 83/83 [00:25<00:00,  3.28it/s]
Epoch: 8
100%|███████████████████████████████████████████| 83/83 [00:25<00:00,  3.28it/s]
Epoch: 9
100%|███████████████████████████████████████████| 83/83 [00:25<00:00,  3.26it/s]
Epoch: 10
100%|███████████████████████████████████████████| 83/83 [00:25<00:00,  3.28it/s

100%|███████████████████████████████████████████| 83/83 [00:23<00:00,  3.49it/s]
Epoch: 91
100%|███████████████████████████████████████████| 83/83 [00:23<00:00,  3.49it/s]
Epoch: 92
100%|███████████████████████████████████████████| 83/83 [00:23<00:00,  3.49it/s]
Epoch: 93
100%|███████████████████████████████████████████| 83/83 [00:23<00:00,  3.59it/s]
Epoch: 94
100%|███████████████████████████████████████████| 83/83 [00:23<00:00,  3.58it/s]
Epoch: 95
100%|███████████████████████████████████████████| 83/83 [00:23<00:00,  3.52it/s]
Epoch: 96
100%|███████████████████████████████████████████| 83/83 [00:23<00:00,  3.56it/s]
Epoch: 97
100%|███████████████████████████████████████████| 83/83 [00:23<00:00,  3.54it/s]
Epoch: 98
100%|███████████████████████████████████████████| 83/83 [00:24<00:00,  3.43it/s]
Epoch: 99
100%|███████████████████████████████████████████| 83/83 [00:23<00:00,  3.53it/s]
Epoch: 100
100%|███████████████████████████████████████████| 83/83 [00:23<00:00,  3.55it/s]
Epoch: 1

100%|███████████████████████████████████████████| 83/83 [00:22<00:00,  3.69it/s]
Epoch: 269
100%|███████████████████████████████████████████| 83/83 [00:22<00:00,  3.65it/s]
Epoch: 270
100%|███████████████████████████████████████████| 83/83 [00:22<00:00,  3.73it/s]
Epoch: 271
100%|███████████████████████████████████████████| 83/83 [00:22<00:00,  3.71it/s]
Epoch: 272
100%|███████████████████████████████████████████| 83/83 [00:22<00:00,  3.68it/s]
Epoch: 273
100%|███████████████████████████████████████████| 83/83 [00:22<00:00,  3.70it/s]
Epoch: 274
100%|███████████████████████████████████████████| 83/83 [00:22<00:00,  3.68it/s]
Epoch: 275
100%|███████████████████████████████████████████| 83/83 [00:22<00:00,  3.75it/s]
Epoch: 276
100%|███████████████████████████████████████████| 83/83 [00:22<00:00,  3.75it/s]
Epoch: 277
100%|███████████████████████████████████████████| 83/83 [00:22<00:00,  3.67it/s]
Epoch: 278
100%|███████████████████████████████████████████| 83/83 [00:22<00:00,  3.71it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



100%|███████████████████████████████████████████| 83/83 [00:23<00:00,  3.59it/s]
Epoch: 494
100%|███████████████████████████████████████████| 83/83 [00:22<00:00,  3.77it/s]
Epoch: 495
100%|███████████████████████████████████████████| 83/83 [00:22<00:00,  3.72it/s]
Epoch: 496
100%|███████████████████████████████████████████| 83/83 [00:22<00:00,  3.73it/s]
Epoch: 497
100%|███████████████████████████████████████████| 83/83 [00:21<00:00,  3.78it/s]
Epoch: 498
100%|███████████████████████████████████████████| 83/83 [00:21<00:00,  3.81it/s]
Epoch: 499
100%|███████████████████████████████████████████| 83/83 [00:21<00:00,  3.80it/s]
